# Import requirements

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
%matplotlib inline

# Scrape historical data from footballdb.com (2010 - 2017)

In [2]:
def generate_url_string(position,year,week):
    '''
    generate url string, position = QB, RB, WR, TE or DST, year = int, week = int
    '''
    str1 = 'http://www.footballdb.com/fantasy-football/index.html?pos='
    str2 = '&yr='
    str3 = '&wk='
    str4 = '&rules=2'
    if position not in ['QB','RB','WR','TE','DST']:
        return "ERROR: position must be 'QB','RB','WR','TE', or'DST'"
    yr_str = str(year)
    wk_str = str(week)
    output_string = str1+position+str2+yr_str+str3+wk_str+str4
    return output_string

def get_year_week_df(url_string):
    '''
    return a dataframe of weekly stats for a given url
    '''
    page = requests.get(url_string)
    soup = BeautifulSoup(page.content, 'html.parser')
    table_headers = soup.find('tr', {'class': 'header right'}).get_text().split('\n')
    column_labels = [str(header).replace('*','') for header in table_headers if len(str(header))>0]
    stats_table = soup.find('table', {'class': 'statistics'})
    row_stats = stats_table.findAll('tr', {'class': ['row0','row1']})
    total_data = []
    if len(row_stats) > 1:
        for row in row_stats:
            player_team = str(row.find('span', {'class':'hidden-xs'}).get_text())
            game_data = [td.text.replace(u'\xa0',' ').encode('utf-8').strip() for td in row.find_all('td') if td.text]
            data = [player_team] + game_data[1:]
            total_data.append(data)
        df = pd.DataFrame(total_data)
        df.columns = column_labels
        return df
    else:
        pass

In [ ]:
for year in range(2010,2018,1):
    for week in range(1,18,1):
        for position in ['QB','RB','WR','TE','DST']:
            #print (position, year, week)
            url_string = generate_url_string(position, year, week)
            df = get_year_week_df(url_string)
            if df is None:
                break
            else:
                csv_filename = './all_data/footballdb_data/'+position +'_'+str(year)+'_week'+str(week)+'.csv'
                df.to_csv(csv_filename)

In [3]:
url_string = generate_url_string('QB', 2013, 5)
df = get_year_week_df(url_string)
df.head()

,Player,Opp,Pts,Att,Cmp,Yds,TD,Int,2Pt,Att,Yds,TD,2Pt,Rec,Yds,TD,2Pt,FL,TD
0,"Tony Romo, Dal",Den,50.00,36,25,506,5,1,1,1,7,0,0,0,0,0,0,0,0
1,"Peyton Manning, Den",@Dal,44.00,42,33,414,4,1,0,4,-8,1,0,0,0,0,0,0,0
2,"Russell Wilson, Sea",@Ind,27.00,31,15,210,2,1,0,13,102,0,0,0,0,0,0,1,0
3,"Geno Smith, NYJ",@Atl,27.00,20,16,199,3,0,0,3,21,0,0,0,0,0,0,0,0
4,"Jay Cutler, Chi",NO,27.00,33,24,358,2,0,0,4,27,0,0,0,0,0,0,1,0


In [5]:
print url_string

http://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2013&wk=5&rules=2


# Historical Fantasy Projections 
# fantasydata.com
# (2013 - 2017)

In [43]:
def get_stats_urls(position, year, week):
    
    if position not in ['QB','RB','WR','TE','DST']:
        return "ERROR: position must be 'QB','RB','WR','TE', or'DST'"
    
    # dictionaries to map strings to values
    position_dict = {'QB':2,'RB':3,'WR':4,'TE':5,'DST':6}
    year_dict = {2013:4, 2014:3, 2015:2, 2016:1, 2017:0}
    position = position_dict[position]
    year = year_dict[year]
    week = week - 1

    variable =  "https://fantasydata.com/nfl-stats/nfl-fantasy-football-stats.aspx?"
    variable += "fs=3&stype=0&sn=%s&scope=1&w=%s&ew=%s&s=&t=0&p=%s&st=" % (year, week, week, position)
    variable += "FantasyPointsDraftKings&d=1&ls=FantasyPointsDraftKings&live=false&pid=true&minsnaps=4"
    return variable

print get_stats_urls('WR',2017,1)

https://fantasydata.com/nfl-stats/nfl-fantasy-football-stats.aspx?fs=3&stype=0&sn=1&scope=1&w=0&ew=0&s=&t=0&p=4&st=FantasyPointsDraftKings&d=1&ls=FantasyPointsDraftKings&live=false&pid=true&minsnaps=4


In [ ]:
https://fantasydata.com/nfl-stats/nfl-fantasy-football-stats.aspx?

fs=3&stype=0&sn=1&scope=1&w=4&ew=4&s=&t=0&p=2&st=

FantasyPointsDraftKings&d=1&ls=FantasyPointsDraftKings&live=false&pid=true&minsnaps=4

In [31]:
def get_projection_urls(position, year, week):
    
    if position not in ['QB','RB','WR','TE','DST']:
        return "ERROR: position must be 'QB','RB','WR','TE', or'DST'"
    
    # dictionaries to map strings to values
    position_dict = {'QB':1,'RB':2,'WR':3,'TE':4,'DST':6}
    year_dict = {2013:4, 2014:3, 2015:2, 2016:1, 2017:0}
    position = position_dict[position]
    year = year_dict[year]
    week = week - 1

    variable =  "https://fantasydata.com/nfl-stats/fantasy-football-weekly-projections.aspx?"
    variable += "fs=3&stype=0&sn=%s&scope=1&w=%s&ew=%s&s=&t=0&p=%s&st=" % (year, week, week, position)
    variable += "FantasyPointsDraftKings&d=1&ls=FantasyPointsDraftKings&live=false&pid=false&minsnaps=4"
    return variable

print get_projection_urls('QB',2015,10)

https://fantasydata.com/nfl-stats/fantasy-football-weekly-projections.aspx?fs=3&stype=0&sn=2&scope=1&w=9&ew=9&s=&t=0&p=1&st=FantasyPointsDraftKings&d=1&ls=FantasyPointsDraftKings&live=false&pid=false&minsnaps=4


# Historical DraftKings Salaries

In [ ]:
# week 9, 2016, RB
url_string = "http://rotoguru1.com/cgi-bin/fyday.pl?week=9&year=2016&game=dk#RB"

position = 'RB'
week = 9


In [ ]:
page = requests.get(url_string)
soup = BeautifulSoup(page.content, 'html.parser')
table_headers = soup.find('tr', {'class': 'header right'}).get_text().split('\n')